In [104]:
import os
from dotenv import load_dotenv  # pip install python-dotenv
import psycopg2  # pip install psycopg2-binary
from sqlalchemy import create_engine  # pip install sqlalchemy
import sys, locale

# print(sys.getdefaultencoding(), locale.getpreferredencoding())
 
# Charge le fichier .env
load_dotenv()
 
# Variables simples depuis .env
host = os.getenv('DB_HOST')      # ex: localhost
port = os.getenv('DB_PORT')      # ex: 5432
dbname = os.getenv('DB_NAME')    # ex: biblio_test
user = os.getenv('DB_USER')      # ex: postgres
password = os.getenv('DB_PASSWORD')  # ton mot de passe
# options="-c client_encoding=UTF8"
 
 
print("Variables chargees depuis .env")
print(f"Connexion à : {host}:{port}/{dbname}")
print(f"Avec  user et mot de passe: {user}:{password}")


Variables chargees depuis .env
Connexion à : localhost:5444/python
Avec  user et mot de passe: postgres:admin123


In [105]:
try:
    # Cree la connexion psycopg2 (adaptateur direct)
    connexion = psycopg2.connect(
        host=host,
        port=port,
        dbname=dbname,
        user=user,
        password=password
    )
    
    # Verification immediate de la connexion
    if connexion.closed == 0:
        print("✓ CONNEXION REUSSIE !")
        print(f"Statut connexion : {'Ouverte' if not connexion.closed else 'Fermée'}")
    else:
        print("✗ Connexion fermée")
    
    # Cursor = "pointeur" pour executer SQL
    curseur = connexion.cursor()
    
    # Test simple : version PostgreSQL
    # curseur.execute("SELECT version();")
    # resultat = curseur.fetchone()
    # print("\n✓ REQUETE EXECUTEE AVEC SUCCES")
    # print(f"PostgreSQL : {resultat[0][:80]}...")  # Affiche les 80 premiers caractères
    
    # Test table biblio
    curseur.execute("SELECT COUNT(*) FROM etudiant;")
    nb_etudiants = curseur.fetchone()[0]
    print(f"\n✓ Table 'etudiant' accessible : {nb_etudiants} etudiants charges")
    
    # Ferme proprement
    curseur.close()
    connexion.close()
    print("\n✓ Connexion fermée proprement")
    
except psycopg2.OperationalError as erreur:
    print("✗ ERREUR CONNEXION : verifie .env/PostgreSQL")
    print(erreur)
except Exception as e:
    print(f"✗ ERREUR : {e}")

✓ CONNEXION REUSSIE !
Statut connexion : Ouverte

✓ Table 'etudiant' accessible : 25 etudiants charges

✓ Connexion fermée proprement


In [106]:
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker, relationship, DeclarativeBase
from sqlalchemy import Column, Integer, String, Date, Float, ForeignKey
from datetime import date, datetime

# Base pour modèles ORM
Base = declarative_base()
 
# Modèle Etudiant (comme ta table)
class Etudiant(Base):
    __tablename__ = 'etudiant'  # Doit correspondre au nom réel de la table
    
    idetudiant = Column(Integer, primary_key=True)  # Clé primaire
    nom = Column(String, nullable=False)
    prenom = Column(String)
    email = Column(String, unique=True)
    date_inscription = Column(Date, default=date.today) # format 2026-01-01
    solde_amende = Column(Float, default=0.0)

    emprunt= relationship("Emprunt", back_populates="etudiant", cascade="delete")

# Modèle Livre
class Livre(Base):
    __tablename__ = 'livre'  # Doit correspondre au nom réel de la table
    
    isbn = Column(String, primary_key=True)  # Clé primaire
    titre = Column(String, nullable=False)
    auteur = Column(String)
    editeur = Column(String)
    annee_publication = Column(Integer)
    exemplaires_dispo = Column(Integer, default=1)

    emprunt= relationship("Emprunt", back_populates="livre", cascade="delete")

# Modèle Emprunt
class Emprunt(Base):
    __tablename__ = 'emprunt'  # Doit correspondre au nom réel de la table
    
    idemprunt = Column(Integer, primary_key=True)  # Clé primaire
    dateemprunt = Column(Date, default=date.today) # format 2026-01-01
    dateretour = Column(Date)
    amende = Column(Float, default=0.0)

    id_etudiant = Column(Integer, ForeignKey('etudiant.idetudiant'))
    id_livre = Column(String, ForeignKey('livre.isbn'))

    etudiant= relationship("Etudiant", back_populates="emprunt")
    livre= relationship("Livre", back_populates="emprunt")
 
# Chaîne connexion SQLAlchemy (utilise psycopg2 en backend)
engine = create_engine(f"postgresql://{user}:{password}@{host}:{port}/{dbname}")

try:
    # Crée session ORM (comme un "cursor avancé")
    Session = sessionmaker(bind=engine)
    session = Session()
    
    # Test : Charge TOUS les étudiants (ORM)
    etudiants = session.query(Etudiant).all()  
    # for etu in etudiants :
    #     print(f" {etu.prenom} {etu.nom} (ID: {etu.idetudiant})")


    # Test : Charge TOUS les livres
    livres = session.query(Livre).all()  
    # for livre in livres :
    #     print(f" {livre.titre} (ISBN: {livre.isbn})")

     # Test : Charge TOUS les Emprunts
    emprunts = session.query(Emprunt).all()  
    for emprunt in emprunts :
        print(f" Emprunt ID: {emprunt.idemprunt},date emprunt: {emprunt.dateemprunt}, Date retour: {emprunt.dateretour}, Étudiant ID: {emprunt.id_etudiant},Etudiant nom : {emprunt.etudiant.nom} , Livre ISBN: {emprunt.id_livre}, Livre titre: {emprunt.livre.titre} ")
    
    # print("ORM -- SUCCESS! Étudiants :")
    # for etu in etudiants[:5]:  # Top 5
    #     print(f"  - {etu.prenom} {etu.nom} (ID: {etu.idetudiant}, email : {etu.email})")
    
    # Exemple requête filtrée (paramétrée auto)
    # retardataires = session.query(Etudiant).limit(3).all()
    # print(f"3 étudiants (exemple filtre) : {len(retardataires)}")
    
    # Ferme session
    session.close()
    
except Exception as e:
    print(f"Erreur ORM : {e}")
finally:
    print("Fin TP !")


 Emprunt ID: 1,date emprunt: 2024-01-05, Date retour: 2024-01-12, Étudiant ID: 1,Etudiant nom : John , Livre ISBN: 9780000000001, Livre titre: Python 
 Emprunt ID: 2,date emprunt: 2024-01-06, Date retour: 2024-01-10, Étudiant ID: 2,Etudiant nom : James , Livre ISBN: 9780000000002, Livre titre: Java 
 Emprunt ID: 3,date emprunt: 2024-01-07, Date retour: 2024-01-08, Étudiant ID: 3,Etudiant nom : Aba , Livre ISBN: 9780000000003, Livre titre: C 
 Emprunt ID: 4,date emprunt: 2024-01-08, Date retour: 2024-01-15, Étudiant ID: 4,Etudiant nom : Ada , Livre ISBN: 9780000000004, Livre titre: C++ 
 Emprunt ID: 5,date emprunt: 2024-01-09, Date retour: 2024-01-20, Étudiant ID: 18,Etudiant nom : Clea , Livre ISBN: 9780000000003, Livre titre: C 
 Emprunt ID: 6,date emprunt: 2024-01-10, Date retour: 2024-01-19, Étudiant ID: 6,Etudiant nom : fifi , Livre ISBN: 9780000000006, Livre titre: PHP 
 Emprunt ID: 7,date emprunt: 2024-01-11, Date retour: 2024-01-21, Étudiant ID: 7,Etudiant nom : sol , Livre ISBN

C:\Users\Alla-EddineBOUKABOU\AppData\Local\Temp\ipykernel_9464\2777093183.py:7: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [ ]:
def create_etu(session, nom, prenom, email, date_inscription,solde_amende):
    try:
        etu = Etudiant(nom=nom,
                       prenom=prenom,
                       email=email,
                       date_inscription=date_inscription,
                       solde_amende=solde_amende)
        session.add(etu)
        session.commit()
        return etu.idetudiant
    except Exception as e:
        session.rollback()
        print(f"Erreur lors de la création de l'étudiant: {e}")
        return None
 
def read_etu(session, nom):
    try:
        return session.query(Etudiant).filter_by(nom=nom).first()
    except Exception as e:
        print(f"Erreur lors de la lecture de l'étudiant: {e}")
        return None
 
def update_etu(session, idetudiant, nouveau_prenom):
    try:
        etu = session.query(Etudiant).get(idetudiant)
        if etu:
            etu.prenom = nouveau_prenom
            session.commit()
            return True
        return False
    except Exception as e:
        session.rollback()
        print(f"Erreur lors de la mise à jour de l'étudiant: {e}")
        return False
 
def delete_etu(session, nom):
    try:
        etu = read_etu(session, nom)
        if etu:
            session.delete(etu)
            session.commit()
            return True
        return False
    except Exception as e:
        session.rollback()
        print(f"Erreur lors de la suppression de l'étudiant: {e}")
        return False

# ----------------------------------------------------------------------


def create_emprunt(session):
    try:
        id_etudiant = input("Entrez l'ID de l'étudiant : ")
        # Check user existant
        check_user = session.query(Etudiant).get(id_etudiant)
        if not check_user:
                print("User non trouvé.") 
                check = False
                while not check:
                    idemprunt = int(input("Veuillez entrer l'ID d'un user existant ou '-1' pour quitter : "))
                    if idemprunt == -1:
                        return False
                    check_user = session.query(Etudiant).get(idemprunt)
                    if check_user:
                        check = True
                    else:
                        print("User non trouvé.")

        # Check le livre existant
        id_livre = input("Entrez l'ISBN du livre : ")
        check_livre = session.query(Livre).get(id_livre)
        if not check_livre:
                print("Livre non trouvé.") 
                check = False
                while not check:
                    id_livre = input("Veuillez entrer l'ISBN d'un livre existant ou 'q' pour quitter : ")
                    if id_livre.lower() == 'q':
                        return
                    check_livre = session.query(Livre).get(id_livre)
                    if check_livre:
                        check = True
                    else:
                        print("Livre non trouvé.")


        # Check la date d'emprunt
        dateemprunt = input("Entrez la date d'emprunt (YYYY-MM-DD) : ")
        if not check_date_format(dateemprunt):
            check = False
            while not check:
                dateemprunt = input("Format de date invalide. Veuillez entrer une date au format YYYY-MM-DD ou 'q' pour quitter : ")
                if dateemprunt.lower() == 'q':
                    return
                if check_date_format(dateemprunt):
                    check = True

        # Check la date de retour
        dateretour = input("Entrez la date de retour (YYYY-MM-DD) ou laissez vide : ")
        if dateretour == '' :
            dateretour = None
        
        elif dateretour!='' and not check_date_format(dateretour):
            check = False
            while not check:
                dateretour = input("Format de date invalide. Veuillez entrer une date au format YYYY-MM-DD ou 'q' pour quitter : ")
                if dateretour.lower() == 'q':
                    return
                if check_date_format(dateretour):
                    check = True

        # Check l'amende
        amende = input("Entrez l'amende (0.0 si aucune) : ")
        if amende == None or amende =='':
            amende = 0.0

        elif not check_float_input(float(amende), 1000.0):
                    check = False
                    while not check:
                        amende = float(input("Valeur invalide. Veuillez entrer une amende entre 0.0 et 1000.0 ou '-1' pour quitter : "))
                        if amende== -1:
                            return False
                        if check_float_input(amende, 1000.0):

                            check = True

        emprunt = Emprunt(id_etudiant=id_etudiant,
                         id_livre=id_livre,
                         dateemprunt=dateemprunt,
                         dateretour=dateretour,
                         amende=float(amende))
        
        # ajout de l'emprunt si tout est ok
        session.add(emprunt)
        session.commit()

        # affichage de l'emprunt créé
        print(f' Emprunt créé avec ID: {emprunt.idemprunt} ')
        new = session.query(Emprunt).get(emprunt.idemprunt)
        if new:
            print(f' Emprunt ID: {new.idemprunt}, Étudiant ID: {new.id_etudiant}, Livre ISBN: {new.id_livre}, date emprunt: {new.dateemprunt}, date retour: {new.dateretour}, amende: {new.amende} ')
        else:
            print("Erreur lors de la création de l'emprunt.")
    except Exception as e:
        session.rollback()
        print(f"Erreur lors de la création de l'emprunt: {e}")

# create_emprunt(session, 1, '9780000000008', date(2024, 6, 1), None, 0.0)

def check_date_format(date_text):
    try:
        datetime.strptime(date_text, '%Y-%m-%d')
        return True
    except ValueError:
        return False

def emprunts_par_date_emprunt(session):
    try:
        dateemprunt = input("Entrez la date d'emprunt (YYYY-MM-DD) : ")
        if not check_date_format(dateemprunt):
            check = False
            while not check:
                dateemprunt = input("Format de date invalide. Veuillez entrer une date au format YYYY-MM-DD ou 'q' pour quitter : ")
                if dateemprunt.lower() == 'q':
                    return
                if check_date_format(dateemprunt):
                    check = True

                    
        liste = session.query(Emprunt).filter_by(dateemprunt=dateemprunt).all()
        if liste:
            print(f"La liste des emprunts pour la date {dateemprunt}:")
            for el in liste:
                print(f" Emprunt ID: {el.idemprunt}, Date emprunt: {el.dateemprunt}, Étudiant ID: {el.id_etudiant},Etudiant nom : {el.etudiant.nom} , Livre ISBN: {el.id_livre}, Livre titre: {el.livre.titre}, ammende: {el.amende} ")
        else:
            print(f"Aucun emprunt trouvé pour la date {dateemprunt}.")
    except Exception as e:
        print(f"Erreur lors de la recherche des emprunts par date: {e}")

# emprunts_par_date_emprunt(session)

def emprunt_par_date_retour(session):
    try:
        dateretour = input("Entrez la date de retour (YYYY-MM-DD) : ")
        if not check_date_format(dateretour):
            check = False
            while not check:
                dateretour = input("Format de date invalide. Veuillez entrer une date au format YYYY-MM-DD ou 'q' pour quitter : ")
                if dateretour.lower() == 'q':
                    return
                if check_date_format(dateretour):
                    check = True

        liste = session.query(Emprunt).filter_by(dateretour=dateretour).all()
        if liste:
            print(f"La liste des emprunts pour la date de retour {dateretour}:")
            for el in liste:
                
                print(f" Emprunt ID: {el.idemprunt}, Étudiant ID: {el.id_etudiant},Etudiant nom : {el.etudiant.nom} , Livre ISBN: {el.id_livre}, Livre titre: {el.livre.titre}, ammende: {el.amende} ")
        else:
            print(f"Aucun emprunt trouvé pour la date de retour {dateretour}.")
    except Exception as e:
        print(f"Erreur lors de la recherche par date de retour: {e}")

# emprunt_par_date_retour(session)

def livres_non_retournes(session):
    try:
        liste = session.query(Emprunt).filter_by(dateretour=None).all()
        if liste:
            print(f" Livres non retournés :")
            for el in liste:
                print(f"{el.idemprunt}, Date retour: {el.dateretour}, Étudiant ID: {el.id_etudiant},Etudiant nom : {el.etudiant.nom} , Livre ISBN: {el.id_livre}, Livre titre: {el.livre.titre}, ammende: {el.amende} ")
        else:
            print("Tous les livres ont été retournés.")
    except Exception as e:
        print(f"Erreur lors de la recherche des livres non retournés: {e}")

# livres_non_retournes(session)

def display_emprunts():
    try:
        emprunts = session.query(Emprunt).all() 
        if emprunts:
            print("Liste de tous les emprunts :") 
            for emprunt in emprunts :
                print(f" Emprunt ID: {emprunt.idemprunt}, Étudiant ID: {emprunt.id_etudiant},Etudiant nom : {emprunt.etudiant.nom} , Livre ISBN: {emprunt.id_livre}, Livre titre: {emprunt.livre.titre}, ammende: {emprunt.amende} ")
        else:
            print("Aucun emprunt trouvé.")
    except Exception as e:
        print(f"Erreur lors de l'affichage des emprunts: {e}")

def check_float_input(float, limit):
    try:
        val = float
        if val <= 0 or val > limit:
            return False
        return True
    except ValueError:
        return False

def emprunt_by_id(session):
    try:
        idemprunt = input("Entrez l'ID à chercher : ")
        emprunt = session.query(Emprunt).get(idemprunt)
        if not emprunt:
            print("Emprunt non trouvé.") 
            check = False
            while not check:
                idemprunt = input("Veuillez entrer un ID d'emprunt valide ou 'q' pour quitter : ")
                if idemprunt.lower() == 'q':
                    return False
                emprunt = session.query(Emprunt).get(idemprunt)
                if emprunt:
                    check = True
                else:
                    print("Emprunt non trouvé.")

        else:
            print("Emprunt trouvé :")
            print(f' Emprunt ID: {emprunt.idemprunt}, Étudiant ID: {emprunt.id_etudiant},Etudiant nom : {emprunt.etudiant.nom} , Livre ISBN: {emprunt.id_livre}, Livre titre: {emprunt.livre.titre}, ammende: {emprunt.amende} ')
    except Exception as e:
        print(f"Erreur lors de la recherche de l'emprunt par ID: {e}")

def delete_emprunt(session):
    try:
        idemprunt = input("Entrez l'ID de l'emprunt à supprimer : ")
        emprunt = session.query(Emprunt).get(idemprunt)
        done= False
        while not done:
            if not emprunt:
                print("Emprunt non trouvé.") 
                check = False
                while not check:
                    idemprunt = input("Veuillez entrer un ID d'emprunt existant ou 'q' pour quitter : ")
                    if idemprunt.lower() == 'q':
                        return False
                    emprunt = session.query(Emprunt).get(idemprunt)
                    if emprunt:
                        check = True
                    else:
                        print("Emprunt non trouvé.")
        
            else:
                print("Emprunt trouvé.")
                session.delete(emprunt)
                session.commit()
                print(f' Emprunt ID: {idemprunt} supprimé avec succès. ')
                done= True
                # return True
    except Exception as e:
        session.rollback()
        print(f"Erreur lors de la suppression de l'emprunt: {e}")

def update_amende(session):
    try:
        idemprunt = input("Entrez l'ID de l'emprunt à mettre à jour : ")
        emprunt = session.query(Emprunt).get(idemprunt)
        done= False
        while not done:
            if not emprunt:
                print("Emprunt non trouvé.") 
                check = False
                while not check:
                    idemprunt = input("Veuillez entrer un ID d'emprunt valide ou 'q' pour quitter : ")
                    if idemprunt.lower() == 'q':
                        return False
                    emprunt = session.query(Emprunt).get(idemprunt)
                    if emprunt:
                        check = True
                    else:
                        print("Emprunt non trouvé.")
        
            else:
                print("Emprunt trouvé.")
                nouvelle_amende = float(input("Entrez la nouvelle amende (0.0 - 1000.0) : "))

                if not check_float_input(nouvelle_amende, 1000.0):
                    check = False
                    while not check:
                        nouvelle_amende = float(input("Valeur invalide. Veuillez entrer une amende entre 0.0 et 1000.0 ou '-1' pour quitter : "))
                        if nouvelle_amende== -1:
                            return False
                        if check_float_input(nouvelle_amende, 1000.0):
                            check = True

                emprunt.amende = nouvelle_amende
                session.commit()

                updated = session.query(Emprunt).get(emprunt.idemprunt)
                if updated:
                    print(f' Nouvelle valeur:') 
                    print(f' Emprunt ID: {updated.idemprunt}, amende: {updated.amende} ')
                else:
                    print("Erreur lors de la mise à jour de l'ammende de l'emprunt.")
                done= True
                # return True
    except Exception as e:
        session.rollback()
        print(f"Erreur lors de la mise à jour de l'amende: {e}")

# update_ammende(session)


# ----------------------------
# def update_date_retour(session, idemprunt, nouvelle_amende):
    
    
# etu = read_etu(session, "Aline")
# if etu:
#     print(etu.idetudiant, etu.prenom, etu.nom, etu.email)
# else:
#     print("Pas trouvé")

In [ ]:
def menu_crud():
    try:
        print("\n=== MENU CRUD EMPRUNT ===")
        print("1: Créer un emprunt")
        print("2: Afficher tous les emprunts")
        print("3: Avoir un emprunt par ID")
        print("4: Mettre à jour l'amende d'un emprunt")
        print("5: Rechercher les emprunts par date d'emprunt")
        print("6: Rechercher les emprunts par date de retour")
        print("7: Afficher les livres non retournés")
        print("8: Supprimer un emprunt")
        print("0: Quitter")
        return input("Choix (0-8): ")

    except Exception as e:
        print(f"Erreur dans le menu: {e}")
        return None
 
# BOUCLE PRINCIPALE
try:
    while True:
        choix = menu_crud()
        
        if choix is None:
            continue
        
        if choix == "1":  
            create_emprunt(session)
        
        elif choix == "2":  
           display_emprunts()     
                
        elif choix == "3": 
            emprunt_by_id(session)
        
        elif choix == "4": 
          update_amende(session)
                
                
        elif choix == "5": 
            emprunts_par_date_emprunt(session)

        elif choix == "6": 
            emprunt_par_date_retour(session)
        
        elif choix == "7": 
            livres_non_retournes(session)
        
        elif choix == "8": 
            delete_emprunt(session)
        
        elif choix == "0": # Fermeture du Code
            print("Au revoir !")
            break
        else:
            print("---------------------------------------------------")
            print("Choix invalide. Veuillez réessayer.")
            print("---------------------------------------------------")

except Exception as e:
    print(f"Erreur dans la boucle principale: {e}")
finally:
    session.close()


=== MENU CRUD EMPRUNT ===
1: Créer un emprunt
2: Afficher tous les emprunts
3: Avoir un emprunt par ID
4: Mettre à jour l'amende d'un emprunt
5: Rechercher les emprunts par date d'emprunt
6: Rechercher les emprunts par date de retour
7: Afficher les livres non retournés
8: Supprimer un emprunt
0: Quitter
Choix invalide. Veuillez réessayer.

=== MENU CRUD EMPRUNT ===
1: Créer un emprunt
2: Afficher tous les emprunts
3: Avoir un emprunt par ID
4: Mettre à jour l'amende d'un emprunt
5: Rechercher les emprunts par date d'emprunt
6: Rechercher les emprunts par date de retour
7: Afficher les livres non retournés
8: Supprimer un emprunt
0: Quitter
Choix invalide. Veuillez réessayer.

=== MENU CRUD EMPRUNT ===
1: Créer un emprunt
2: Afficher tous les emprunts
3: Avoir un emprunt par ID
4: Mettre à jour l'amende d'un emprunt
5: Rechercher les emprunts par date d'emprunt
6: Rechercher les emprunts par date de retour
7: Afficher les livres non retournés
8: Supprimer un emprunt
0: Quitter
Au revo